In [1]:
import os
import xarray as xr
import dask
from dask.diagnostics import ProgressBar
import hydra
from omegaconf import DictConfig
import logging
import numpy as np

In [2]:
import hydra
from omegaconf import DictConfig
# 初始化 Hydra
hydra.initialize(config_path="conf", version_base=None)
# 组合配置
cfg = hydra.compose(config_name="config_uv_1")
cfg

{'grib_store_path': '/datasets/grib2_data', 'hdf5_store_path': '/datasets/hdf5_data_uvsp', 'dt': 1, 'start_train_year': 2015, 'end_train_year': 2015, 'test_years': [2015], 'out_of_sample_years': [2015], 'compute_mean_std': True, 'variables': ['10m_u_component_of_wind', '10m_v_component_of_wind', 'surface_pressure']}

In [3]:
	all_years = [2015]
	# Return the Zarr paths
	grib_paths = []
	for year in all_years:
		for month in range(8,10):
			grib_path = f"{cfg.grib_store_path}/{year}/wnd10m.cdas1.{year}{month:02d}.grb2"
			grib_paths.append(grib_path)

In [5]:
grib_arrays = [xr.open_dataset(path) for path in grib_paths]
grib_arrays

[<xarray.Dataset> Size: 244MB
 Dimensions:            (time: 124, step: 6, latitude: 200, longitude: 205)
 Coordinates:
   * time               (time) datetime64[ns] 992B 2015-08-01 ... 2015-08-31T1...
   * step               (step) timedelta64[ns] 48B 01:00:00 02:00:00 ... 06:00:00
     heightAboveGround  float64 8B ...
   * latitude           (latitude) float64 2kB 41.81 41.6 41.4 ... 1.329 1.124
   * longitude          (longitude) float64 2kB 98.18 98.39 98.59 ... 139.7 139.9
     valid_time         (time, step) datetime64[ns] 6kB ...
 Data variables:
     u                  (time, step, latitude, longitude) float32 122MB ...
     v                  (time, step, latitude, longitude) float32 122MB ...
 Attributes:
     GRIB_edition:            2
     GRIB_centre:             60
     GRIB_centreDescription:  National Center for Atmospheric Research 
     GRIB_subCentre:          1
     Conventions:             CF-1.7
     institution:             National Center for Atmospheric Resear

In [8]:
	cfsr_xarray_concat = xr.concat(
		grib_arrays, dim="time"
	)

In [10]:
	cfsr_xarray_channel = xr.concat(
		[cfsr_xarray_concat[z] for z in list(cfsr_xarray_concat.data_vars.keys())], dim="channel"
	)
	cfsr_xarray_stacked =  cfsr_xarray_channel.stack(newtime=['time', 'step'])
	cfsr_xarray_stacked = cfsr_xarray_stacked.swap_dims({'newtime':'valid_time'})
	cfsr_xarray_drop = cfsr_xarray_stacked.drop_vars('newtime')
	cfsr_xarray = cfsr_xarray_drop.transpose("valid_time", "channel", "latitude", "longitude")
	cfsr_xarray.name = "fields"
	cfsr_xarray = cfsr_xarray.astype("float32")

In [11]:
cfsr_xarray

<xarray.DataArray 'fields' (valid_time: 1464, channel: 2, latitude: 200,
                            longitude: 205)> Size: 480MB
array([[[[ 6.55,  6.49,  6.23, ...,  2.2 ,  2.24,  2.21],
         [ 5.55,  5.44,  5.19, ...,  2.43,  2.63,  2.69],
         [ 4.  ,  3.98,  3.92, ...,  2.67,  3.11,  3.45],
         ...,
         [-0.97, -1.35, -1.46, ..., -1.69, -1.57, -1.51],
         [-0.64, -1.11, -1.4 , ..., -1.82, -1.73, -1.72],
         [-0.41, -0.87, -1.28, ..., -1.9 , -1.86, -1.89]],

        [[-8.83, -8.77, -8.51, ..., -0.85, -0.84, -0.64],
         [-8.71, -8.41, -7.93, ..., -1.29, -1.28, -1.01],
         [-7.82, -7.43, -6.93, ..., -1.65, -1.68, -1.45],
         ...,
         [-0.96, -1.1 , -1.03, ...,  0.3 ,  0.48,  0.67],
         [-0.93, -1.28, -1.39, ...,  0.4 ,  0.54,  0.7 ],
         [-0.86, -1.29, -1.5 , ...,  0.43,  0.53,  0.66]]],


       [[[ 5.93,  6.05,  5.96, ...,  2.49,  2.5 ,  2.46],
         [ 4.97,  5.07,  5.02, ...,  2.77,  2.93,  2.96],
         [ 3.48,  3.67,  3.81, ...,  2.98,  3.39,  3.67],
...
         [-3.47, -2.87, -2.14, ..., -0.57, -0.56, -0.57],
         [-3.45, -3.08, -2.41, ..., -0.66, -0.67, -0.69],
         [-3.21, -2.99, -2.38, ..., -0.66, -0.68, -0.7 ]]],


       [[[ 0.9 ,  1.29,  1.65, ...,  0.27,  0.38,  0.57],
         [ 1.06,  1.23,  1.37, ...,  0.69,  0.85,  1.02],
         [ 1.14,  1.02,  0.9 , ...,  1.06,  1.24,  1.42],
         ...,
         [ 1.65,  1.12,  0.33, ..., -3.69, -3.5 , -3.31],
         [ 1.52,  1.07,  0.32, ..., -3.71, -3.52, -3.31],
         [ 1.12,  0.79,  0.16, ..., -3.69, -3.5 , -3.28]],

        [[ 1.18,  0.94,  0.77, ...,  7.05,  6.03,  4.72],
         [ 0.68,  0.46,  0.38, ...,  7.25,  6.45,  5.26],
         [ 1.19,  1.05,  1.02, ...,  7.49,  6.89,  6.11],
         ...,
         [-3.01, -2.45, -1.79, ..., -0.32, -0.32, -0.33],
         [-3.  , -2.69, -2.12, ..., -0.48, -0.51, -0.54],
         [-2.88, -2.76, -2.28, ..., -0.54, -0.58, -0.62]]]], dtype=float32)
Coordinates:
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 2kB 41.81 41.6 41.4 ... 1.329 1.124
  * longitude          (longitude) float64 2kB 98.18 98.39 98.59 ... 139.7 139.9
  * valid_time         (valid_time) datetime64[ns] 12kB 2015-08-01T01:00:00 ....
    time               (valid_time) datetime64[ns] 12kB 2015-08-01 ... 2015-0...
    step               (valid_time) timedelta64[ns] 12kB 01:00:00 ... 06:00:00
Dimensions without coordinates: channel
Attributes: (12/30)
    GRIB_paramId:                             131
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      41000
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                U component of wind
    GRIB_shortName:                           u
    GRIB_units:                               m s**-1
    long_name:                                U component of wind
    units:                                    m s**-1
    standard_name:                            eastward_wind